<a href="https://colab.research.google.com/github/akimotolab/Policy_Optimization_Tutorial/blob/main/ex1_evolutionary_policy_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題内容

ここでは，`1_policy_optimization_introduction.ipynb`で学んだ内容を発展させ，「より優れた方策を獲得する方法」を検討し，実験的に評価しましょう．

そのために，次のようなステップで検討していきます．
1. 「より優れた方策を獲得できる方法」という曖昧な目的を明確にしましょう．
2. 明確になった目的を評価するための方法を検討しましょう．
3. 各々方法を検討し，実験を実施し，比較しましょう．

# 0. 準備

`1_policy_optimization_introduction.ipynb`から必要なコードをコピーしてきます．

## 0.1 インストール＆インポート

In [ ]:
# 必要なパッケージのインストール
!apt update
!pip install swig
!apt install xvfb
!pip install pyvirtualdisplay
!pip install gymnasium[box2d]

In [ ]:
from pyvirtualdisplay import Display
import torch

# 仮想ディスプレイの設定
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

# gpuが使用される場合の設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [ ]:
import random
import numpy as np
from scipy.special import softmax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import gymnasium as gym
from IPython import display

## 0.2 関数やクラスの定義

* `rollout`：与えられた方策を用いて環境とインタラクションし，状態行動履歴を返す関数
* `visualize`：`rollout`において`render=True`とした場合に返される画像からアニメーションを作成する関数
* `cumulative_reward`：`rollout`において返される履歴から累積報酬を計算する関数

In [ ]:
def rollout(envname, policy=None, render=False, seed=None):
    if render:
        env = gym.make(envname, render_mode="rgb_array")
    else:
        env = gym.make(envname)
    history = []
    img = []

    # 乱数の設定
    if seed is not None:
        random.seed(int(seed))
    envseed = random.randint(0, 1000)
    actseed = random.randint(0, 1000)
    observation, info = env.reset(seed=envseed)
    env.action_space.seed(actseed)

    # 可視化用の設定
    if render:
        d = Display()
        d.start()
        img.append(env.render())

    # メインループ（環境とのインタラクション）
    terminated = False
    truncated = False
    while not (terminated or truncated):

        # 行動を選択
        if policy is None:
            action = env.action_space.sample()
        else:
            action = policy(observation)

        # 行動を実行
        next_observation, reward, terminated, truncated, info = env.step(action)
        history.append([observation, action, next_observation, reward, terminated, truncated, info])
        observation = next_observation
        if render:
            display.clear_output(wait=True)
            img.append(env.render())
    env.close()
    return history, img


def visualize(img):
    dpi = 72
    interval = 50
    plt.figure(figsize=(img[0].shape[1]/dpi, img[0].shape[0]/dpi), dpi=dpi)
    patch = plt.imshow(img[0])
    plt.axis=('off')
    animate = lambda i: patch.set_data(img[i])
    ani = animation.FuncAnimation(plt.gcf(), animate, frames=len(img), interval=interval)
    display.display(display.HTML(ani.to_jshtml()))
    plt.close()


def cumulative_reward(history):
    return sum(hist[3] for hist in history)

`Cmaes`クラス
* `ask`：新しい解（方策パラメータ）を一つだけ生成する関数
* `tell`：`ask`で生成された解（方策パラメータ）の目的関数値（累積報酬のマイナス1倍）を与えると，次の解を生成するための正規分布の平均ベクトルや共分散行列を更新する関数

In [ ]:
class Cmaes:
    """(1+1)-Active-CMA-ES [Arnold 2010] with Simplified 1/5th Success Rule"""

    def __init__(self, init_fx, init_x, init_s):
        self.dim = len(init_x)

        self.aup = np.exp(2.0 / (2.0 + self.dim))
        self.adown = self.aup ** (-0.25)
        self.pthresh = 0.44
        self.cp = 1.0 / 12.0
        self.cc = 2.0 / (self.dim + 2)
        self.ccovp = 2.0 / (self.dim**2 + 6)
        self.ccovm = 0.4 / (self.dim**1.6 + 1)

        self.x = np.array(init_x, copy=True)
        self.xc = np.zeros(self.dim)
        self.yc = np.zeros(self.dim)
        self.zc = np.zeros(self.dim)
        self.s = init_s
        self.psucc = 0.5
        self.fhist = [init_fx]
        self.p = np.zeros(self.dim)
        self.chol = np.eye(self.dim)
        self.cholinv = np.eye(self.dim)

    def ask(self):
        self.zc = np.random.randn(self.dim)
        self.yc = np.dot(self.chol, self.zc)
        self.xc = self.x + self.s * self.yc
        return self.xc

    def tell(self, fx):
        if fx <= self.fhist[0]:
            self.fhist = [fx] + self.fhist[:min(4, len(self.fhist))]
            self.x = self.xc
            self.s *= self.aup
            self.psucc = (1.0 - self.cp) * self.psucc + self.cp
            if self.psucc > self.pthresh:
                self.p = (1.0 - self.cc) * self.p
                d = self.ccovp * (1.0 - self.cc * (2.0 - self.cc))
            else:
                self.p = (1.0 - self.cc) * self.p + np.sqrt(self.cc * (2.0 - self.cc)) * self.yc
                d = self.ccovp
            w = np.dot(self.cholinv, self.p)
            wnorm2 = np.dot(w, w)
            if wnorm2 > 1e-6:
                a = np.sqrt(1.0 - d)
                b = np.sqrt(1.0 - d) * (np.sqrt(1.0 + self.ccovp * wnorm2 / (1.0 - d)) - 1.0) / wnorm2
                self.chol = a * self.chol + b * np.outer(self.p, w)
                self.cholinv = self.cholinv / a - (b / (a**2 + a * b * wnorm2)) * np.outer(w, np.dot(w, self.cholinv))
        else:
            self.s *= self.adown
            self.psucc = (1.0 - self.cp) * self.psucc
            if len(self.fhist) > 4 and fx > self.fhist[4] and self.psucc <= self.pthresh:
                znorm2 = np.dot(self.zc, self.zc)
                ccov = self.ccovm if 1.0 >= self.ccovm * (2.0 * znorm2 - 1.0) else 1.0 / (2.0 * znorm2 - 1.0)
                a = np.sqrt(1.0 + ccov)
                b = np.sqrt(1.0 + ccov) * (np.sqrt(1.0 - ccov * znorm2 / (1.0 + ccov)) - 1) / znorm2
                self.chol = a * self.chol + b * np.outer(self.yc, self.zc)
                self.cholinv = self.cholinv / a - (b / (a**2 + a * b * znorm2)) * np.outer(self.zc, np.dot(self.zc, self.cholinv))

Neural Network方策

* 環境を変えた場合，NsやNaも変更することが必要
* K はモデルの自由度をコントロールするパラメータ

In [ ]:
class NNPolicy:
    def __init__(self, weights, Na, Ns, K):
        self.B = weights[:K]
        self.W = weights[K:K*Ns+K].reshape((K, Ns))
        self.V = weights[K*Ns+K:K*Ns+K+K*Na].reshape((Na, K))

    def __call__(self, observation):
        z = np.dot(observation, self.W.T)
        z += self.B
        z = softmax(z)
        z = np.dot(z, self.V.T)
        return 2 * np.tanh(z)

パーセンタイル目的関数

*  `NNPolicy`の方策パラメータを引数にとり，10回の独立したエピソードを実行し，得られた累積報酬の90パーセンタイルを出力する関数

In [ ]:
envname = "Pendulum-v1"
Ns = 3
Na = 1
K = 20
N = (Ns + Na + 1) * K

def objective(x):
    policy = NNPolicy(x, Na, Ns, K)
    seed_array = np.arange(100, 1100, 100)
    return_array = np.zeros(len(seed_array))
    for i, seed in enumerate(seed_array):
        history, img = rollout(envname, policy=policy, render=False, seed=seed)
        return_array[i] = cumulative_reward(history)
    return np.percentile(- return_array, 90) + 1e-10 * np.dot(x, x)

## 0.3 実行スクリプト

In [ ]:
# (1+1)-CMA-ESの作成
sigma = 10
mean = sigma * np.random.randn(N)
f_hist = np.empty(1000)  # この長さが最大のイテレーション数
f_hist[0] = objective(mean)
es = Cmaes(f_hist[0], mean, sigma)

# 最適化の実行
for t in range(len(f_hist)):
    w = es.ask()
    f_hist[t] = objective(w)
    es.tell(f_hist[t])
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        # 300 イテレーション通して改善が見られない場合には終了
        f_hist = f_hist[:t+1]
        break

# 最適化途中で得られた解の目的関数値を可視化
plt.plot(f_hist)

# 1. 目的を明確にする



「より優れた方策を獲得できる方法」という表現は曖昧なので，何を持って優れていると判断するかの指標を定量的に定めることが必要です．指標は目的によって変わるものであるため，目的に沿った合理的な指標を定めることが重要です．


## 1.1 方策の評価指標


ここでは，上述の`objective`関数で評価している指標と同じく，「方策によって得られる累積報酬の90パーセンタイルの値」を方策の良さとして定義します．なお，この評価方法は強化学習の文脈では必ずしも一般的なものではありません．強化学習はあくまで累積報酬の期待値を最大化することを目的とするため，この目的のためにはあまり適さないことになります．進化計算を用いるからこそ可能な指標であることに注意してください．

さて，`objective`関数では，固定された乱数10個を用いてそれぞれ1エピソード分のインタラクションを実施し，累積報酬を計算しています．これは最適化の都合上の設計ですが，評価の観点からは適切ではありません．その理由として，以下の２つが挙げられます．

1. 10エピソード分の累積報酬（10個のサンプル）で90パーセンタイルを計算しても，その推定精度（理論上の90パーセンタイルに対する10サンプルで計算される推定値の差）は高いとは言えない．

2. 関数内で使用されている乱数系列にオーバーフィットした方策が得られており，他の乱数系列で評価した場合にはより悪い90%になる可能性がある．

そのため，評価時には最適化時よりも高い精度での90%の推定が求められます．
そのための方法としてはいくつか考えられますが，ここでは「最適化時とは異なる100エピソード分の累積報酬値を計算し，それらの90パーセンタイルを計算した結果」を評価指標とします．

In [ ]:
def evaluate(policy, envname):
    return_array = np.zeros(100)
    for i in range(len(return_array)):
        history, img = rollout(envname, policy=policy, render=False)
        return_array[i] = cumulative_reward(history)
    return np.percentile(- return_array, 90), return_array

上の実行例で得られた方策を評価してみます．最適化時に得られた目的関数の値よりも高い値になってしまっていることが確認できます．

In [ ]:
policy = NNPolicy(es.x, Na, Ns, K)
p90, return_array = evaluate(policy, envname)
print(p90)

経験分布関数も確認しておきましょう．

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
plt.grid()

## 1.2 方法の評価



「良い方策」の指標が得られたので，「良い最適化法」の評価指標を検討します．
当然良い方策を得ることが目的ですから，より良い方策を得ることが可能な最適化法が優れているということになります．

ただし，以下の点に注意が必要です．
1. 最適化法を実行するたびに得られる解（方策）が変わる（最適化法自体の確率的な要因，初期値の確率的な要因）
2. 最適化法によって，最適化にかかるコスト（ここでは環境とのインタラクション数がボトルネックですので，これをコストと考えます）が異なる

一点目に対処するために，最低でも5回程度は同じ最適化を，乱数を変えて実行しましょう．

二点目はなかなか難しい問題です．早くある程度良い解を見つけることができる方法と，時間はかかるもののよりよい解を見つけることができる方法のどちらがよいかは，どれだけのコストを許容できるかに依存します．そのため，いくつかのアプローチが取られています．

1. 収束したとみなせるほど十分なコストをかけた後に得られる解同士の性能比較
2. 十分に良い性能と判断できる性能のしきい値に達するまでに費やしたコスト同士の比較
3. 収束曲線を描き，多元的に比較

いずれも良し悪しあります．１つ目はコストを気にせずとにかく良い解が得られるのかの評価です．２つ目は同じ性能に達するまでにかかるコストの比較なので，これが可能なのであれば概ね妥当な評価になります．ただし，十分に良い性能と判断できるしきい値の設定に依存してしまいます．３つ目は定量的な評価になりませんが，速度と性能の両者を考慮した比較が可能になります．必ずしも勝者が決まるような比較ではありませんが，方法間の性質の比較を行う際には有益です．ここでは３つ目の方法を採用することにしましょう．




# ２．実験方法の検討

評価方法が決まったので，比較実験の方法を実装していきましょう．
比較対象となるアルゴリズムは各自で実装してもらうことになります．


今回用いている`Pendulum-v1`環境では，1エピソード毎に実行されるインタラクション数が500で固定です．そのため，コストは単純にエピソード数と考えて問題ありません．また，上で定義した目的関数は一度の90パーセンタイルの計算に10エピソードを必要とします．今回の(1+1)-CMA-ESでは，1イテレーションに1度の目的関数評価を必要とするため，
(1+1)-CMA-ESの1イテレーション当たりのコストは10エピソード，となります．アルゴリズムによってこのコストが変わりますので，平等な比較になるように注意が必要です．


比較したい方法を以下のようなかたちで実装しておきます．これは上に示した(1+1)-CMA-ESにほかなりません．ただし，評価方法に合わせてログの取り方を変えています．

In [ ]:
def optimize1(maxeps=10000, logfile='./1p1cmaes.txt'):
    # (1+1)-CMA-ESの作成
    neps = 10
    sigma = 10
    mean = sigma * np.random.randn(N)
    f_hist = np.empty(maxeps // 10)  # 最適化中の目的関数値のログ
    f_hist[:] = np.NAN
    f_hist[0] = objective(mean)
    es = Cmaes(f_hist[0], mean, sigma)

    # 最適化の途中で得られた解を10イテレーション（100エピソード）毎にファイルに書き出し
    with open(logfile, 'w') as f:
        f.write(str(neps) + ' ' + str(f_hist[0]) + ' ' + ' '.join(map(str, es.x)) + '\n')

    # 最適化の実行
    for t in range(1, len(f_hist)):
        w = es.ask()
        f_hist[t] = objective(w)
        neps += 10
        es.tell(f_hist[t])

        # 最適化の途中で得られた解を10イテレーション（100エピソード）毎にファイルに書き出し
        if (t+1) % 10 == 0:
            print(t+1, np.min(f_hist[:t+1]), es.s)
            with open(logfile, 'a') as f:
                f.write(str(neps) + ' ' + str(f_hist[0]) + ' ' + ' '.join(map(str, es.x)) + '\n')

    return es.x

1.1，1.2節で示した方法で最適化法を評価するために，乱数を変えて最適化を5回繰り返し，その結果を保存します．保存された解を`evaluate`関数を用いて評価することで，それぞれの最適化試行の途中に得られた解を評価します．最後に，5試行分の平均と標準偏差を確認します．

In [ ]:
# 最適化実行
# ローカルで実行する場合には，複数試行の実施は並列に実施してしまうと効率的
for trial in range(1, 6):
    print("Trial ", trial)
    np.random.seed(trial * 1234)
    optimize1(maxeps=10000, logfile='./1p1cmaes_trial{}.txt'.format(trial))

In [ ]:
# 方策の評価
# 方策の評価も比較的時間がかかるので，ローカルで並列実行するとよい
hist = []
for trial in range(1, 6):
    logfile = './1p1cmaes_trial{}.txt'.format(trial)
    dat = np.loadtxt(logfile)
    p90_array = np.zeros((2, len(dat)))
    for i in range(len(dat)):
        p90_array[0, i] = dat[i, 0]
        p90_array[1, i] = evaluate(NNPolicy(dat[i, 1:], Na, Ns, K), envname)[0]
    hist.append(p90_array)

In [ ]:
# 学習曲線の可視化
import matplotlib.pyplot as plt
plt.figure()
# 各試行の結果をプロット
for i, p90_array in enumerate(hist):
    plt.plot(p90_array[0], p90_array[1], ':')
# 平均と標準偏差をプロット
p90_stacked = np.vstack([p90_array[1] for p90_array in hist])
p90_avg = np.mean(p90_stacked, axis=0)
p90_std = np.std(p90_stacked, axis=0)
plt.plot(hist[0][0], p90_avg, '-', color='k')
plt.fill_between(hist[0][0], p90_avg - p90_std, p90_avg + p90_std, alpha=0.3, color='k')
plt.grid()
# 図の保存
plt.savefig('1p1cmaes.eps')

方法間の比較にはこの平均と標準偏差を主に見ていくことになります．各試行での結果もここでは確認していますが，これは平均や標準偏差が5試行を十分に代表していると考えられるかを確認するためです．試行毎にあまりにも大きな偏りがある場合，平均をみることが適切とは言えないので，一度は確認しておくことが重要です．問題なければみやすさのために比較の際にはプロットしないようにしましょう．

２つ以上の結果を比較したい場合には，一つの図にして保存しましょう．
比較をしたいのに複数の図にまたがっていると比較が難しくなります．
また，データを可視化して保存する場合には，ベクトル形式の図（epsなど）で保存しましょう．ビットマップ形式で保存するとデータの情報が欠落してしまうため，科学技術文書では望ましくありません．

# 3 アルゴリズムの提案と実験的比較

実験の準備は整ったので，あとは(1+1)-CMA-ESよりも優れた方法を検討し，実装しましょう．方法といっても，完全に新しい方法を設計する必要はありません．(1+1)-CMA-ESをベースにして，一部を変更する，といった方法で結構です．抜本的に異なる方法を検討するよりも，一部のみが異なる方法を比較したほうが，どういった工夫がどのように影響するのかを理解しやすいためです（研究の基本です）．

結果的に優れているとみなせない方法になっても構いません．ただ，時間の許す限り，試行錯誤してみてください．設計した方法を`optimize1`をまねて`optimize2`などとして実行できる関数として実装しましょう．その後，上の方法に従って比較しましょう．

以下に改善案の例を挙げておきます．

* 最適化時の目的関数を90パーセンタイルから95パーセンタイルに変更する
  - 理由と期待される効果：最適化時に得られている目的関数値と比較して評価時の値が著しく悪いのは，最適化時に用いている乱数系列にオーバーフィットしてしまっているからであると考えられる．ただ，評価時の経験分布関数を見ると，完全に最適化時の乱数系列だけにオーバーフィットしているのではなく，望ましい性能を示す割合が90％よりも低くなってしまっている．そこで，最適化時の目的関数を90パーセンタイルでなく95パーセンタイルや99パーセンタイルなどとしておくことで，この割合が下がったとしても90パーセント以上の試行で優れた性能を示す方策を獲得できるのではないか，と期待される．

* 最適化時の目的関数評価におけるエピソード数を100に変更する
  - 理由と期待される効果：上述のように，オーバーフィットにより，最適化している目的関数の最小化が評価時の真の目的に対応していない点に改善の余地がある．目的関数はあくまで真の目的の推定値であるから，この推定精度をあげることができれば，乖離を抑えることが可能であると考えられる．目的関数計算時のエピソード数を増加させることで，これは実現可能であると考えられる．しかし，一度の目的関数評価毎に費やすエピソード数が増加するため，最終性能は改善するかもしれないが，コストは多くかかるようになると予想される．

上のように，原因となりそうな部分に着目し，これを改善することを考えてみてください．また，その変更による影響を予め予想しておくことが重要です．実験は，この**予想（仮説）が正しいかどうかを検証するプロセス**です．



# 4. レポートの提出

レポートは次のような章立てで，LaTeXなどを用いて作成してください．

1. 実験の目的，検証したい仮説．実験は仮説検証のプロセスですから，必ず目的から書きましょう．目的があって初めて適切な実験設定があります．

2. 提案するアルゴリズム

  1. 提案法のアイディア，提案の理由と期待される効果（上の記述を参考にしてください）

  2. 提案法の具体的な方法．(1+1)-CMA-ESをベースとした方法であれば，アルゴリズムの全体の説明がなくとも，変更点が明確であれば構いませんが，他の人が見てその方法を実装するのに必要な情報を全て記載しましょう．研究論文では再現性といって，重要な評価項目となります．

3. 実験方法．この項目も他の人が再現できるように必要な情報を全て記載しましょう．

  1. 評価指標の説明
  2. 実験設定や具体的な実験方法．使用した環境（'Pendulum-v1'）であること，使用した方策，など，その実験を再現するために必要な情報を記載しましょう．
  
4. 実験結果．結果として表示する図の読み取り方，図から読み取れること，これが予想と一致しているのかについての結論を述べましょう．

5. 考察．今回の結論が得られた原因についての考察を記述しましょう．良い考察は，**単なる予想ではなく，それを裏付ける結果とともに示されているもの**になります．
例えば方法Aと方法Bを比較した場合に何らかの違いが性能差として見られたとします．この差は，方法AとBの間のアルゴリズム上の差によって，例えば目的関数の推定精度が変わり，その結果として性能に差が生じていると考えられます．性能の差を見ているだけでは，アルゴリズムの差→推定精度の差，という部分は予想に過ぎないことになります．そこで，性能差だけでなく，推定精度に差が生じているのか，を実際にデータをとって確認することにより，推論に根拠を与えていきます．

6. まとめ．今回の実験を振り返り，得られた知見をまとめましょう．また，現在の実験では確認しきれていないこと，検証の弱いところ，考えられる改善点，などについてまとめましょう．



